In [10]:
'''
Métodos de la clase:
    url: str
	date: datetime.datetime
	content: str
	renderedContent: str
	id: int
	user: 'User'
	replyCount: int
	retweetCount: int
	likeCount: int
	quoteCount: int
	conversationId: int
	lang: str
	source: str
	sourceUrl: typing.Optional[str] = None
	sourceLabel: typing.Optional[str] = None
	links: typing.Optional[typing.List['TextLink']] = None
	media: typing.Optional[typing.List['Medium']] = None
	retweetedTweet: typing.Optional['Tweet'] = None
	quotedTweet: typing.Optional['Tweet'] = None
	inReplyToTweetId: typing.Optional[int] = None
	inReplyToUser: typing.Optional['User'] = None
	mentionedUsers: typing.Optional[typing.List['User']] = None
	coordinates: typing.Optional['Coordinates'] = None
	place: typing.Optional['Place'] = None
	hashtags: typing.Optional[typing.List[str]] = None
	cashtags: typing.Optional[typing.List[str]] = None
	card: typing.Optional['Card'] = None
'''

"\nMétodos de la clase:\n    url: str\n\tdate: datetime.datetime\n\tcontent: str\n\trenderedContent: str\n\tid: int\n\tuser: 'User'\n\treplyCount: int\n\tretweetCount: int\n\tlikeCount: int\n\tquoteCount: int\n\tconversationId: int\n\tlang: str\n\tsource: str\n\tsourceUrl: typing.Optional[str] = None\n\tsourceLabel: typing.Optional[str] = None\n\tlinks: typing.Optional[typing.List['TextLink']] = None\n\tmedia: typing.Optional[typing.List['Medium']] = None\n\tretweetedTweet: typing.Optional['Tweet'] = None\n\tquotedTweet: typing.Optional['Tweet'] = None\n\tinReplyToTweetId: typing.Optional[int] = None\n\tinReplyToUser: typing.Optional['User'] = None\n\tmentionedUsers: typing.Optional[typing.List['User']] = None\n\tcoordinates: typing.Optional['Coordinates'] = None\n\tplace: typing.Optional['Place'] = None\n\thashtags: typing.Optional[typing.List[str]] = None\n\tcashtags: typing.Optional[typing.List[str]] = None\n\tcard: typing.Optional['Card'] = None\n"

In [11]:
#!pip install sntwitter
import snscrape.modules.twitter as sntwitter
import pandas as pd
# query: filter: links -> con links en el tweet
# query: near:Madrid within:15km

query = "#ods igualdad filter:links"
tweets = []
limit = 10000

for tweet in sntwitter.TwitterSearchScraper(query).get_items():
    
    # print(vars(tweet))
    # break
    if len(tweets) == limit:
        break
    else:
        tweets.append([tweet.date, tweet.user,tweet.lang,tweet.coordinates ,tweet.content, tweet.likeCount, tweet.retweetCount,tweet.replyCount, tweet.url])
        
df = pd.DataFrame(tweets, columns=['Date', 'User','Lang','coordinates', 'Tweet', 'Likes','Retweets' , 'Replies' , 'Url'])

In [12]:
#df.sort_values(by="Likes", ascending=False)

In [13]:
#df.Lang.value_counts()

In [14]:
#sacamos el huso horario para que sea compatible con excel
import datetime as dt
df['Date'] = df['Date'].dt.tz_localize(None)

In [15]:
import nltk
from nltk.sentiment.vader import SentimentIntensityAnalyzer
nltk.download("vader_lexicon")
### Analizando el sentimiento con VADER
sia = SentimentIntensityAnalyzer()
# aplicamoc vader para asignar un valor positivo, negativo, neutro y un compuesto que integra todos
df["neg"] = df["Tweet"].apply(lambda x:sia.polarity_scores(x)["neg"])
df["pos"] = df["Tweet"].apply(lambda x:sia.polarity_scores(x)["pos"])
df["neu"] = df["Tweet"].apply(lambda x:sia.polarity_scores(x)["neu"])
df["compuesto"] = df["Tweet"].apply(lambda x:sia.polarity_scores(x)["compound"])

[nltk_data] Downloading package vader_lexicon to
[nltk_data]     C:\Users\mparada\AppData\Roaming\nltk_data...
[nltk_data]   Package vader_lexicon is already up-to-date!


In [16]:
# Filtramos por likes, retweets y positividad
retweets=1
likes=4
compuesto=0.4
df.loc [(df.Retweets > retweets) & (df.Likes > likes) & (df.compuesto > compuesto)].head(3)

,Date,User,Lang,coordinates,Tweet,Likes,Retweets,Replies,Url,neg,pos,neu,compuesto
142,2022-04-25 16:26:42,https://twitter.com/forgender,es,None,🟣Las #certificaciones a favor de la #Igualdad ...,6,6,0,https://twitter.com/forgender/status/151862757...,0.0,0.074,0.926,0.4019
173,2022-04-08 06:18:44,https://twitter.com/CEIPForoRomano,es,None,El equipo de atención a la diversidad del @cei...,16,3,0,https://twitter.com/CEIPForoRomano/status/1512...,0.0,0.078,0.922,0.4003
290,2022-03-08 15:10:07,https://twitter.com/ikerjimenezm,es,None,La dirección de #ImpulsoGlobal es resultado de...,25,3,1,https://twitter.com/ikerjimenezm/status/150121...,0.0,0.066,0.934,0.4019


In [17]:
#Guardamos en Excel
df.to_excel('sdg_noticias.xlsx')

In [18]:
#df.loc[df['Tweet'].nlargest(1,['Likes'])]
whatsapp = df.nlargest(2,['Likes'])
whatsapp = str(whatsapp.Tweet.to_string())
whatsapp

'2291    Un golpe de realidad: ningún país ha alcanzado...\n2125    Ningún país ha alcanzado la igualdad de género...'

In [19]:
#pip install pywhatkit
#pip install flask

import pywhatkit
#numero_telefono= input("ingresa numero telefono: ")
numero_telefono= "+5491131753300"

pywhatkit.sendwhatmsg(numero_telefono,whatsapp, 14 , 13, 15, True, 2)

In 17660 Seconds WhatsApp will open and after 15 Seconds Message will be Delivered!


In [20]:
#pywhatkit.sendwhatmsg_to_group (id)